In [1]:
#General imports.
import ROOT as r
import math
from BlindersPy3 import Blinders
from BlindersPy3 import FitType
r.gStyle.SetOptStat(0)
r.gStyle.SetOptFit(1111)
import matplotlib.pyplot as plt
import csv
import pandas

import seaborn as sns
sns.set(style="ticks", color_codes=True)
import numpy as np
import matplotlib.pyplot as plt
import matplotlib 
matplotlib.rc('xtick', labelsize=20) 
matplotlib.rc('ytick', labelsize=20) 
import os
import scipy

import uproot # https://indico.cern.ch/event/686641/contributions/2894906/attachments/1606247/2548596/pivarski-uproot.pdf
from awkward import JaggedArray
from root_pandas import read_root # https://github.com/scikit-hep/root_pandas

import warnings
warnings.filterwarnings('once')

%matplotlib inline

Welcome to JupyROOT 6.12/06


This file is meant to loop over all of the files in a given list and draw the crystal arrangements of the outliers / any other specified plots

In [2]:
#import a bunch of useful functions for this analysis
from evwTools import *

In [3]:
#Names of the files to loop over

fileNamesPartial = ["gm2offline_ana_12769487_15932.00176.root",
                    "gm2offline_ana_12769488_15929.00040.root",
                    "gm2offline_ana_12769489_15928.00148.root",
                    "gm2offline_ana_12769490_15926.00236.root",
                    "gm2offline_ana_12769491_15925.00138.root",
                    "gm2offline_ana_12769502_15932.00136.root",
                    "gm2offline_ana_12769503_15925.00302.root",
                    "gm2offline_ana_12769504_15932.00404.root",
                    "gm2offline_ana_12769505_15924.00114.root",
                    "gm2offline_ana_12769506_15926.00463.root",
                    "gm2offline_ana_12769517_15925.00347.root",
                    "gm2offline_ana_12769518_15925.00416.root",
                    "gm2offline_ana_12769519_15923.00286.root",
                    "gm2offline_ana_12769520_15929.00068.root",
                    "gm2offline_ana_12769521_15927.00098.root",
                    "gm2offline_ana_12769532_15926.00341.root",
                    "gm2offline_ana_12769533_15930.00326.root",
                    "gm2offline_ana_12769534_15932.00270.root",
                    "gm2offline_ana_12769535_15926.00098.root",
                    "gm2offline_ana_12769536_15922.00104.root",
                    "gm2offline_ana_12769547_15927.00076.root",
                    "gm2offline_ana_12769548_15931.00241.root",
                    "gm2offline_ana_12769549_15926.00137.root",
                    "gm2offline_ana_12769550_15926.00289.root",
                    "gm2offline_ana_12769551_15925.00292.root",
                    "gm2offline_ana_12769563_15933.00156.root",
                    "gm2offline_ana_12769564_15921.00059.root",
                    "gm2offline_ana_12769565_15930.00026.root",
                    "gm2offline_ana_12769567_15928.00147.root",
                    "gm2offline_ana_12769568_15926.00188.root",
                    "gm2offline_ana_12769579_15932.00352.root",
                    "gm2offline_ana_12769580_15922.00429.root",
                    "gm2offline_ana_12769581_15933.00346.root",
                    "gm2offline_ana_12769582_15930.00193.root",
                    "gm2offline_ana_12769583_15929.00109.root",
                    "gm2offline_ana_12769594_15925.00436.root",
                    "gm2offline_ana_12769595_15926.00212.root",
                    "gm2offline_ana_12769596_15926.00271.root",
                    "gm2offline_ana_12769597_15925.00479.root",
                    "gm2offline_ana_12769598_15931.00313.root",
                    "gm2offline_ana_12769610_15922.00054.root",
                    "gm2offline_ana_12769611_15922.00291.root",
                    "gm2offline_ana_12769612_15922.00098.root",
                    "gm2offline_ana_12769613_15921.00360.root",
                    "gm2offline_ana_12769614_15922.00021.root",
                    "gm2offline_ana_12769627_15926.00409.root",
                    "gm2offline_ana_12769628_15932.00428.root",
                    "gm2offline_ana_12769629_15926.00443.root",
                    "gm2offline_ana_12769630_15926.00345.root",
                    "gm2offline_ana_12769631_15924.00247.root",
                    "gm2offline_ana_12769642_15931.00001.root",
                    "gm2offline_ana_12769643_15922.00156.root",
                    "gm2offline_ana_12769644_15927.00012.root",
                    "gm2offline_ana_12769645_15925.00478.root",
                    "gm2offline_ana_12769646_15927.00436.root",
                    "gm2offline_ana_12769657_15928.00123.root",
                    "gm2offline_ana_12769658_15924.00077.root",
                    "gm2offline_ana_12769659_15933.00299.root",
                    "gm2offline_ana_12769660_15929.00372.root",
                    "gm2offline_ana_12769661_15925.00189.root",
                    "gm2offline_ana_12769672_15928.00105.root",
                    "gm2offline_ana_12769673_15930.00328.root"]

#and their paths.
fileNames = ["./data/dataExternal/Oct29_v10_DataSet/" + x for x in fileNamesPartial]

#uncomment this to just loop over the local file for which we also have the island data
#fileNames = ["./data/gm2offline_ana.root"]

In [4]:
#this will be the cut that will determine whether or not something is printed. Written in root format, not pyroot.
uniqueIDref = 1636900369001090001700150
condition = constructConditionFromUniqueIDComp(uniqueIDref)
print(condition)

 runNum == 16369 && subRunNum == 369 && fill == 109 && caloNum == 17 && islandWest == 150


In [6]:
found = False
for fileName in fileNames:
    try:
        f = r.TFile(fileName)
        twest = f.Get("farline").Get("eventTree")
        teast = f.Get("farline").Get("eastTree")
        tcomp = f.Get("farline").Get("evwTree")
        terr = f.Get("farline").Get("evwNoPartnerTree")
        tcomp2 = f.Get("farline").Get("wveTree")
        terr2 = f.Get("farline").Get("wveNoPartnerTree")
    except:
        print("ERROR: File ", fileName, "not found")
        continue
        
    print("Starting file:", fileName)
    
    dingbefore = 0

    fout = r.TFile("temp_tree_storage.root","recreate")
    tcomp_partial = tcomp.CopyTree(condition)
    print("     Matching entries in this file:", tcomp_partial.GetEntries())
    
    for i, entry in enumerate(tcomp_partial):
        found = True
        print("UniqueID Found in file:", fileName)
        counter = 0
        ding = constructUniqueID(entry)
        if (dingbefore == ding):
            continue

        print("     Unique island: ", ding)
        print("     deltaE: ", entry.deltaE)

        crystalsEast = [x for x in entry.crystalsEast]

        teast_partial = teast.CopyTree( constructConditionFromUniqueIDEast(ding) )
        for k, entry3 in enumerate(teast_partial):
            dong = constructUniqueIDeast(entry3)
            if ding == dong:
                print("          Found another island with this unique ID. Adding their crystals together:")
                print("               ",crystalsEast)
                crystalsEast += [x for x in entry3.crystalsEast]
                print("               ",crystalsEast)

        crystalsWest = []
        twest_partial = twest.CopyTree( constructConditionFromUniqueIDWest(ding) )
        for j, entry2 in enumerate(twest_partial):
            dong = constructUniqueIDwest(entry2)
            if (ding == dong):
                counter = counter + 1
                print("          Found Match (", i, j, counter,")")
                print("              East: ", crystalsEast)
                print("              West: ", [x for x in entry2.crystalsWest])
                crystalsWest += [x for x in entry2.crystalsWest]

        imshowCompare( toMap(crystalsEast), toMap(crystalsWest), True, True, str(ding) )
        dingbefore = ding
        
    if(found):
        break
        
if(not found):
    print("ERROR: UniqueID", uniqueIDref, "not found")

Starting file: ./data/dataExternal/Oct29_v10_DataSet/gm2offline_ana_12769487_15932.00176.root
     Matching entries in this file: 0
Starting file: ./data/dataExternal/Oct29_v10_DataSet/gm2offline_ana_12769488_15929.00040.root
     Matching entries in this file: 0
Starting file: ./data/dataExternal/Oct29_v10_DataSet/gm2offline_ana_12769489_15928.00148.root
     Matching entries in this file: 0
Starting file: ./data/dataExternal/Oct29_v10_DataSet/gm2offline_ana_12769490_15926.00236.root
     Matching entries in this file: 0
Starting file: ./data/dataExternal/Oct29_v10_DataSet/gm2offline_ana_12769491_15925.00138.root
     Matching entries in this file: 0
Starting file: ./data/dataExternal/Oct29_v10_DataSet/gm2offline_ana_12769502_15932.00136.root
     Matching entries in this file: 0
Starting file: ./data/dataExternal/Oct29_v10_DataSet/gm2offline_ana_12769503_15925.00302.root
     Matching entries in this file: 0
Starting file: ./data/dataExternal/Oct29_v10_DataSet/gm2offline_ana_12769504